In [17]:
import os
from canvas_data.api import CanvasDataAPI
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import pathlib
import json
import shutil
import petl as etl
from tqdm import tqdm

API_KEY = ""
API_SECRET =""

# Extract
Get the tables from Canvas Data Portal using Canvas API

In [18]:
class Extract():

    def __init__(self, API_KEY, API_SECRET):
        from tqdm import tqdm
        self.API_KEY = API_KEY
        self.API_SECRET = API_SECRET


    def connect_canvas_api(self):
        from canvas_data.api import CanvasDataAPI
        cd = CanvasDataAPI(api_key=self.API_KEY, api_secret=self.API_SECRET, download_chunk_size=1024*1024)
        return cd
    
    def unzip_files(self, directory = '/Users/larissatrasvina/thesis-canvas/src/ETL/downloads'):
        dir_name = 'unzipped'
        new_dir = pathlib.Path(directory, dir_name)
        new_dir.mkdir(parents=True, exist_ok=True)
        for filename in tqdm(os.listdir(directory)):
            try: 
                filepath = os.path.join(directory, filename)
                with gzip.open(filepath, 'r+') as file_in:
                    with open(f'{directory}/unzipped/{filename[:-3]}.txt', 'wb') as file_out:
                        shutil.copyfileobj(file_in, file_out)
            except:
                print(f'Could not unzip file {filename[:-3]}')
        

    def extract_files(self):
        cd = self.connect_canvas_api()
        try: 
            files = cd.download_files(dump_id='latest', include_requests=False)
            unzip = self.unzip_files()
        except Exception as e: 
            print(e)
    
    
    def get_schema(self):
        cd = self.connect_canvas_api()
        #dumps = cd.get_dumps()
        #files_nmbr = len(dumps)
        try:
            schema = cd.get_schema('latest', key_on_tablenames=True)
            with open("schema.json", "w") as outfile:
                json.dump(schema, outfile)
            return schema
        except Exception as e: 
            print(e)
    
extraction = Extract(API_KEY, API_SECRET)
extraction.extract_files()
schema = extraction.get_schema()


 57%|█████▋    | 12/21 [00:00<00:00, 57.95it/s]

Could not unzip file unzip


100%|██████████| 21/21 [00:00<00:00, 76.73it/s]


# Transform
In order to make changes in the data, first we have to take a look to find any possible issues.

In [76]:
with open('schema.json', 'r') as file: 
    schema = json.load(file)
    discarded = []
    for table_name in schema:
        cols =  []
        table = schema[table_name]['columns']
        for column in table:
            #cols.update( {'name' : column['name'],'type' : column['type']} )
            try:
                if column['description'][11] == 'Foreign key':
                    print(column['description'])
                    discarded.append(table_name)
            except: 
                continue
            should_avoid = set(discarded)
            cols.append((column['type'], column['name']))
    print(len(should_avoid))

0
